In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

In [2]:
%%time

transactions = pd.read_parquet('transactions_train.parquet')
customers = pd.read_parquet('customers.parquet')
articles = pd.read_parquet('articles.parquet')

CPU times: user 3.22 s, sys: 2.05 s, total: 5.27 s
Wall time: 1.84 s


In [71]:
customers['age_bucket'] = pd.cut(customers['age'].fillna(22), [15, 24, 35, 50, 100], 
                                    labels=['16-24', '25-35', '36-50', '51-100'])

In [164]:
customers.Active.value_counts()

-1    907576
 1    464404
Name: Active, dtype: int64

In [165]:
customers.FN.value_counts()

-1    895050
 1    476930
Name: FN, dtype: int64

In [166]:
customers.club_member_status.value_counts()

 0    1272491
 1      92960
-1       6062
 2        467
Name: club_member_status, dtype: int64

In [3]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0
...,...,...,...,...,...,...
31774722,2020-09-22,18439937050817258297,891591003,0.084729,2,104
31774723,2020-09-22,18439937050817258297,869706005,0.084729,2,104
31779097,2020-09-22,18440902715633436014,918894002,0.016932,1,104
31779098,2020-09-22,18440902715633436014,761269001,0.016932,1,104


In [5]:
VALID = True

if VALID:
    transactions = transactions[transactions.t_dat<'2020-09-16']

In [7]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [8]:
%%time
c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 33.8 s, sys: 2.4 s, total: 36.2 s
Wall time: 33.4 s


In [9]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 660 ms, sys: 75.2 ms, total: 735 ms
Wall time: 733 ms


In [10]:
candidates_last_purchase = transactions.copy()

In [11]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week = weeks

CPU times: user 6.92 s, sys: 24 ms, total: 6.94 s
Wall time: 6.94 s


In [12]:
transactions[transactions.customer_id==28847241659200]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29024918,2020-07-14,28847241659200,730683036,0.042356,1,94
29024919,2020-07-14,28847241659200,851094001,0.011000,1,94
29024920,2020-07-14,28847241659200,757303012,0.030492,1,94
29202087,2020-07-18,28847241659200,762846001,0.025407,1,95
29202088,2020-07-18,28847241659200,829308001,0.033881,1,95
29527049,2020-07-26,28847241659200,887770001,0.016932,1,96
30989557,2020-08-31,28847241659200,760084003,0.025407,1,101
31100629,2020-09-03,28847241659200,925246001,0.128797,2,102


In [13]:
candidates_last_purchase[candidates_last_purchase.customer_id==28847241659200]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29024918,2020-07-14,28847241659200,730683036,0.042356,1,95
29024919,2020-07-14,28847241659200,851094001,0.011000,1,95
29024920,2020-07-14,28847241659200,757303012,0.030492,1,95
29202087,2020-07-18,28847241659200,762846001,0.025407,1,96
29202088,2020-07-18,28847241659200,829308001,0.033881,1,96
29527049,2020-07-26,28847241659200,887770001,0.016932,1,101
30989557,2020-08-31,28847241659200,760084003,0.025407,1,102
31100629,2020-09-03,28847241659200,925246001,0.128797,2,104


In [20]:
transactions = transactions.merge(articles[['article_id','prod_name', 'colour_group_name',
                                            'graphical_appearance_name']],
                                 on='article_id', how='left')

transactions = transactions.merge(customers[['age_bucket', 'customer_id']], on='customer_id', how='left')

In [148]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,prod_name,colour_group_name,graphical_appearance_name,age_bucket
0,2020-07-08,857913002275398,599580068,0.008458,1,94,40,27,3,16-24
1,2020-07-08,857913002275398,776237011,0.025407,1,94,33,7,0,16-24
2,2020-07-08,857913002275398,844294001,0.011847,1,94,12320,27,3,16-24
3,2020-07-08,1658289241058394,877773001,0.007610,1,94,7243,2,1,16-24
4,2020-07-08,3828854365940846,507883009,0.013542,1,94,1535,0,0,36-50


In [ ]:
## From same channel find alternate item from same prod group with highest number of sale
## From same channel find alternate item of lower price with a high number of sale
## From same channel find alternate item of different color with highest number of sale
## From same channel find alternate item of different grpahical appearance with highest number of sale

In [59]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby(['week','sales_channel_id'])['article_id'].value_counts() \
    .groupby(['week','sales_channel_id']).rank(method='dense', ascending=False) \
    .groupby(['week','sales_channel_id']).head(12).rename('bestseller_rank').astype('int8')

In [60]:
sales

week  sales_channel_id  article_id
94    1                 372860002      1
                        372860001      2
                        759871002      3
                        464297007      4
                        817477003      5
                                      ..
103   2                 923758001      8
                        805947001      9
                        751471001     10
                        715624001     11
                        896169005     12
Name: bestseller_rank, Length: 240, dtype: int8

In [85]:
best_sell_df = sales.reset_index()
best_sell_df = best_sell_df.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df['RankType'] = 'MostBought'

In [120]:
## From same channel find alternate item from same prod group with highest number of sale

def find_alternate_items(base_itm, base_channel, base_wk, base_prod):

    item_1 = base_itm
    channel_id = base_channel
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & (transactions.sales_channel_id==channel_id) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [123]:
alternate_best_sell_df = pd.DataFrame(columns = best_sell_df.columns)

wk_list = []
sc_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df)):
    
    bs_itm = best_sell_df['article_id'][i]
    bs_channel = best_sell_df['sales_channel_id'][i]
    bs_wk = best_sell_df['week'][i]
    bs_prod = best_sell_df['prod_name'][i]
    bs_rank = best_sell_df['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items(bs_itm, bs_channel, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    sc_list.extend([bs_channel]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [126]:
alternate_best_sell_df['week'] = wk_list
alternate_best_sell_df['sales_channel_id'] = sc_list
alternate_best_sell_df['article_id'] = art_list
alternate_best_sell_df['bestseller_rank'] = bs_list
alternate_best_sell_df['RankType'] = rt_list

In [133]:
best_sell_df = pd.concat([best_sell_df.drop('prod_name', axis=1), 
                          alternate_best_sell_df.drop('prod_name', axis=1)], axis=0)

best_sell_df = best_sell_df[~(best_sell_df.article_id=='')] \
                .sort_values(['week','sales_channel_id','bestseller_rank'])

In [147]:
best_sell_df.to_parquet('valid_best_sell_with_channel.parquet')

In [ ]:
## From same age_bucket find alternate item from same prod group with highest number of sale
## From same age_bucket find alternate item of lower price with a high number of sale
## From same age_bucket find alternate item of different color with highest number of sale
## From same age_bucket find alternate item of different grpahical appearance with highest number of sale

In [149]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,prod_name,colour_group_name,graphical_appearance_name,age_bucket
0,2020-07-08,857913002275398,599580068,0.008458,1,94,40,27,3,16-24
1,2020-07-08,857913002275398,776237011,0.025407,1,94,33,7,0,16-24
2,2020-07-08,857913002275398,844294001,0.011847,1,94,12320,27,3,16-24
3,2020-07-08,1658289241058394,877773001,0.007610,1,94,7243,2,1,16-24
4,2020-07-08,3828854365940846,507883009,0.013542,1,94,1535,0,0,36-50


In [150]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby(['week','age_bucket'])['article_id'].value_counts() \
    .groupby(['week','age_bucket']).rank(method='dense', ascending=False) \
    .groupby(['week','age_bucket']).head(12).rename('bestseller_rank').astype('int8')

In [152]:
best_sell_df_ab = sales.reset_index()
best_sell_df_ab = best_sell_df_ab.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df_ab['RankType'] = 'MostBought'

In [154]:
## From same channel find alternate item from same prod group with highest number of sale

def find_alternate_items_ab(base_itm, base_age_bucket, base_wk, base_prod):

    item_1 = base_itm
    age_bkt = base_age_bucket
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & (transactions.age_bucket==age_bkt) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [155]:
alternate_best_sell_df_ab = pd.DataFrame(columns = best_sell_df_ab.columns)

wk_list = []
ab_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df_ab)):
    
    bs_itm = best_sell_df_ab['article_id'][i]
    bs_ab = best_sell_df_ab['age_bucket'][i]
    bs_wk = best_sell_df_ab['week'][i]
    bs_prod = best_sell_df_ab['prod_name'][i]
    bs_rank = best_sell_df_ab['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items_ab(bs_itm, bs_ab, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    ab_list.extend([bs_ab]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [156]:
alternate_best_sell_df_ab['week'] = wk_list
alternate_best_sell_df_ab['age_bucket'] = ab_list
alternate_best_sell_df_ab['article_id'] = art_list
alternate_best_sell_df_ab['bestseller_rank'] = bs_list
alternate_best_sell_df_ab['RankType'] = rt_list

In [157]:
best_sell_df_ab = pd.concat([best_sell_df_ab.drop('prod_name', axis=1), 
                          alternate_best_sell_df_ab.drop('prod_name', axis=1)], axis=0)

best_sell_df_ab = best_sell_df_ab[~(best_sell_df_ab.article_id=='')] \
                .sort_values(['week','age_bucket','bestseller_rank'])

In [160]:
best_sell_df_ab.to_parquet('valid_best_sell_with_agebucket.parquet')

In [167]:
## Find alternate item from same prod group with highest number of sale
## Find alternate item of lower price with a high number of sale
## Find alternate item of different color with highest number of sale
## Find alternate item of different grpahical appearance with highest number of sale

In [168]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,prod_name,colour_group_name,graphical_appearance_name,age_bucket
0,2020-07-08,857913002275398,599580068,0.008458,1,94,40,27,3,16-24
1,2020-07-08,857913002275398,776237011,0.025407,1,94,33,7,0,16-24
2,2020-07-08,857913002275398,844294001,0.011847,1,94,12320,27,3,16-24
3,2020-07-08,1658289241058394,877773001,0.007610,1,94,7243,2,1,16-24
4,2020-07-08,3828854365940846,507883009,0.013542,1,94,1535,0,0,36-50


In [169]:
## top 12 articles from last 10 weeks based on weekly sales

sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [170]:
best_sell_df_all = sales.reset_index()
best_sell_df_all = best_sell_df_all.merge(articles[['article_id','prod_name']], on='article_id', how='left')
best_sell_df_all['RankType'] = 'MostBought'

In [172]:
## From same channel find alternate item from same prod group with highest number of sale

def find_alternate_items_all(base_itm, base_wk, base_prod):

    item_1 = base_itm
    wk_num = base_wk
    prod_nm = base_prod
    
    item_2 = ''
    item_3 = ''
    item_4 = ''
    item_5 = ''

    color_1 = articles[articles.article_id==item_1]['colour_group_name'].iloc[0]
    g_1 = articles[articles.article_id==item_1]['graphical_appearance_name'].iloc[0]

    p1 = transactions[(transactions.article_id==item_1) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]['price'].mean()

    filter_df = transactions[(transactions.article_id!=item_1) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]

    if len(filter_df)>0:

        item_2 = filter_df.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
        color_2 = articles[articles.article_id==item_2]['colour_group_name'].iloc[0]
        g_2 = articles[articles.article_id==item_2]['graphical_appearance_name'].iloc[0]

        p2 = transactions[(transactions.article_id==item_2) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num)]['price'].mean()

        p_low = min(p1, p2)
        filter_df_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 (transactions.price < p_low)]

        if len(filter_df_2)>0:

            item_3 = filter_df_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
            color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
            g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            filter_df_3 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.colour_group_name.isin([color_1, color_2, color_3]))]

            if len(filter_df_3)>0:
                item_4 = filter_df_3.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_4 = articles[articles.article_id==item_4]['colour_group_name'].iloc[0]
                g_4 = articles[articles.article_id==item_4]['graphical_appearance_name'].iloc[0]

                filter_df_4 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3, item_4])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3, g_4]))]

                if len(filter_df_4)>0:
                    item_5 = filter_df_4.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]            

            else:
                filter_df_3_1 = transactions[~(transactions.article_id.isin([item_1, item_2, item_3])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.graphical_appearance_name.isin([g_1, g_2, g_3]))]

                if len(filter_df_3_1)>0:
                    item_4 = filter_df_3_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

        else:
            filter_df_2_1 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                 (transactions.prod_name==prod_nm) & 
                 (transactions.week==wk_num) & 
                 ~ (transactions.colour_group_name.isin([color_1, color_2]))]

            if len(filter_df_2_1)>0:
                item_3 = filter_df_2_1.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]
                color_3 = articles[articles.article_id==item_3]['colour_group_name'].iloc[0]
                g_3 = articles[articles.article_id==item_3]['graphical_appearance_name'].iloc[0]

            else:
                filter_df_2_2 = transactions[~(transactions.article_id.isin([item_1, item_2])) & 
                     (transactions.prod_name==prod_nm) & 
                     (transactions.week==wk_num) & 
                     ~ (transactions.graphical_appearance_name.isin([g_1, g_2]))]

                if len(filter_df_2_2)>0:
                    item_3 = filter_df_2_2.groupby('week')['article_id'].value_counts().head(1).index.values[0][1]

    return item_2, item_3, item_4, item_5


In [174]:
alternate_best_sell_df_all = pd.DataFrame(columns = best_sell_df_all.columns)

wk_list = []
art_list = []
bs_list = []
rt_list = []

for i in range(0, len(best_sell_df_all)):
    
    bs_itm = best_sell_df_all['article_id'][i]
    bs_wk = best_sell_df_all['week'][i]
    bs_prod = best_sell_df_all['prod_name'][i]
    bs_rank = best_sell_df_all['bestseller_rank'][i]
    
    a,b,c,d = find_alternate_items_all(bs_itm, bs_wk, bs_prod)

    wk_list.extend([bs_wk]*4)
    art_list.extend([a,b,c,d])
    bs_list.extend([bs_rank]*4)
    rt_list.extend(['SecondMost', 'LowerPrice', 'AnotherColor', 'AnotherAppearance'])

In [175]:
alternate_best_sell_df_all['week'] = wk_list
alternate_best_sell_df_all['article_id'] = art_list
alternate_best_sell_df_all['bestseller_rank'] = bs_list
alternate_best_sell_df_all['RankType'] = rt_list

In [176]:
best_sell_df_all = pd.concat([best_sell_df_all.drop('prod_name', axis=1), 
                          alternate_best_sell_df_all.drop('prod_name', axis=1)], axis=0)

best_sell_df_all = best_sell_df_all[~(best_sell_df_all.article_id=='')] \
                .sort_values(['week','bestseller_rank'])

In [178]:
best_sell_df_all.to_parquet('valid_best_sell_all.parquet')